Il s'agit ici de la fonction QuickCertify qui est la premiere etape de notre methode de Test Systematique de la robustesse des modeles KNN. Ici, a partir de certaines conditions, nous verifions la robustesse. Ces conditions suffisantes sont conçues pour éviter l’étape la plus coûteuse de l’algorithme KNN, qui est la phase d’apprentissage qui s’appuie sur des validations croisées p-fold pour calculer le paramètre K optimal.

In [1]:
import numpy as np
from scipy.spatial import distance

def nearNeighborsSet(data, k, input_x, y=None, n=0):
    # Calculate the distance between input_x and all points in the data
    distances = [(distance.euclidean(input_x, x), x, label) for x, label in data]
    
    # Calculer le removal set 
    if y is not None:
        
        # Sort the distances and get the labels of the k+n nearest neighbors
        if n is not None :
            # near_neighbors = [(elt[1], elt[2]) for elt in sorted(distances)[:k+n]]
            # distances.sort(axis=0)
            # near_neighbors = [(elt[1], elt[2]) for elt in distances[:k+n]]
            # sorted_distances = np.sort(distances, axis=0)
            # near_neighbors = [(elt[1], elt[2]) for elt in sorted_distances[:k+n]]
            sorted_distances = sorted(distances, key=lambda x: x[0])
            near_neighbors = [(elt[1], elt[2]) for elt in sorted_distances[:k+n]]
            # Supprimer n éléments ayant pour label y
            count = 0
            j = len(near_neighbors) - 1  # Commencer par la fin de la liste
            while count < n and j >= 0:
                if near_neighbors[j][1] == y:
                    near_neighbors.pop(j)
                    count += 1
                j -= 1
        else:
            # Erreur dans les parametres de la fonction 
            raise ValueError("Erreur dans les parametres de la fonction")
    else:
        sorted_distances = sorted(distances, key=lambda x: x[0])
        near_neighbors = [(elt[1], elt[2]) for elt in sorted_distances[:k+n]]

    # Return the set of k nearest neighbors
    return near_neighbors

data = [
    ([1.2, 3.5], 'Star'),
    ([2.5, 4.8], 'Star'),
    ([3.7, 2.1], 'Star'),
    ([5.1, 6.2], 'Triangle'),
    ([6.9, 1.8], 'Triangle'),
    ([7.5, 5.5], 'Triangle'),
    ([9.2, 8.1], 'Star'),
    ([8.5, 7.4], 'Triangle'),
    ([0.5, 1.9], 'Star')
]
n = 2
input_x = [5, 4.5]

print(nearNeighborsSet(data, 2, input_x, 'Star', n))

[([5.1, 6.2], 'Triangle'), ([7.5, 5.5], 'Triangle')]


In [2]:
from collections import Counter

def mostFreqLabel(data, k, input_x, y=None, n=0):
    neighbors = nearNeighborsSet(data, k, input_x, y, n)  

    # Get the labels of the neighbors
    labels = [label for point, label in neighbors]

    # Get the most common label
    most_common_label = Counter(labels).most_common(1)[0][0]

    return most_common_label


from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

# Créer une liste de paires (x, y) à partir de X et y
xy = list(zip(X, y))
# print((xy))

# Afficher les 5 premières paires
# print(GenPromisingSubset(data=xy, n=2, x=[5.1, 3.5, 1.4, 0.2], y=0))
print(mostFreqLabel(data=xy, k=1, input_x=[5.1, 3.5, 1.4, 0.2], y='0', n=2))

0


In [3]:
def QuickCertify(data, n, input_x):
    label_set = [] # To store the most frequent labels for each value of K[^1^][1]
    Kset = [1, 2, 3] # Set of K values

    for k in Kset:
        y = mostFreqLabel(data, k, input_x)
        label_set.append(y)
        
        print('---', y, '---')
        
        # Most frequent label after removal[^2^][2]
        y_prime = mostFreqLabel(data, k, input_x, y, n)

        if y != y_prime:
            return False

        # If all the most frequent labels are not the same, return False
        if len(set(label_set)) > 1:
            return False
        
    return True

data = [
    ([1.2, 3.5], 'Star'),
    ([2.5, 4.8], 'Star'),
    ([3.7, 2.1], 'Star'),
    ([5.1, 6.2], 'Triangle'),
    ([6.9, 1.8], 'Triangle'),
    ([7.5, 5.5], 'Triangle'),
    ([9.2, 8.1], 'Star'),
    ([8.5, 7.4], 'Triangle'),
    ([0.5, 1.9], 'Star')
]
n = 2
input_x = [5, 4.5]

print(QuickCertify(data, n, input_x))

--- Triangle ---
False


In [ ]:
import matplotlib.pyplot as plt

data = [
    ([1.2, 3.5], 'Star'),
    ([2.5, 4.8], 'Star'),
    ([3.7, 2.1], 'Star'),
    ([5.1, 6.2], 'Triangle'),
    ([6.9, 1.8], 'Triangle'),
    ([7.5, 5.5], 'Triangle'),
    ([9.2, 8.1], 'Star'),
    ([8.5, 7.4], 'Triangle'),
    ([0.5, 1.9], 'Star')
]

for point in data:
    if point[1] == 'Star':
        plt.scatter(point[0][0], point[0][1], color='red')
    elif point[1] == 'Triangle':
        plt.scatter(point[0][0], point[0][1], color='blue')
    elif point[1] == 'Square':
        plt.scatter(point[0][0], point[0][1], color='green')

plt.show()
